In [2]:
import modulos as md
lista=['MAVUSDT']

lista=md.lista_de_monedas()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol,'30m')
        data = md.estrategia_bb(data,tp_flag = True)
        resultado = md.backtesting(data,plot_flag=True)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {resultado['Profit Factor']}")
    except Exception as falla:
        pass  



C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


MAVUSDT - retorno: 2.0% - trades: 2 - Profit Factor: 4.105250159124188


In [3]:
resultado

Start                     2023-06-28 08:00:00
End                       2023-07-07 00:00:00
Duration                      8 days 16:00:00
Exposure Time [%]                    9.832134
Equity Final [$]                  1020.097424
Equity Peak [$]                   1023.997424
Return [%]                           2.009742
Buy & Hold Return [%]              -30.038421
Return (Ann.) [%]                  106.737975
Volatility (Ann.) [%]               38.223151
Sharpe Ratio                         2.792495
Sortino Ratio                       18.073189
Calmar Ratio                         44.12537
Max. Drawdown [%]                   -2.418971
Avg. Drawdown [%]                   -2.088394
Max. Drawdown Duration        3 days 16:00:00
Avg. Drawdown Duration        2 days 01:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       6.153486
Worst Trade [%]                     -1.498931
Avg. Trade [%]                    